# Testing out spatial extension in duckdb

In [1]:
import duckdb
import pandas as pd

%load_ext sql
conn = duckdb.connect("duckdb/database.ddb")
%sql conn --alias duckdb

## Install spatial extension

In [2]:
%%sql
INSTALL spatial;
LOAD spatial;

Running query in 'duckdb'

Success


## Explore Data

In [9]:
%%sql
SELECT * FROM './pluto/MapPLUTO_UNCLIPPED.shp' LIMIT 1

Running query in 'duckdb'

Borough,Block,Lot,CD,BCT2020,BCTCB2020,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,PolicePrct,HealthCent,HealthArea,Sanitboro,SanitDistr,SanitSub,Address,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,Overlay1,Overlay2,SPDist1,SPDist2,SPDist3,LtdHeight,SplitZone,BldgClass,LandUse,Easements,OwnerType,OwnerName,LotArea,BldgArea,ComArea,ResArea,OfficeArea,RetailArea,GarageArea,StrgeArea,FactryArea,OtherArea,AreaSource,NumBldgs,NumFloors,UnitsRes,UnitsTotal,LotFront,LotDepth,BldgFront,BldgDepth,Ext,ProxCode,IrrLotCode,LotType,BsmtCode,AssessLand,AssessTot,ExemptTot,YearBuilt,YearAlter1,YearAlter2,HistDist,Landmark,BuiltFAR,ResidFAR,CommFAR,FacilFAR,BoroCode,BBL,CondoNo,Tract2010,XCoord,YCoord,ZoneMap,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,FIRM07_FLA,PFIRM15_FL,Version,DCPEdited,Latitude,Longitude,Notes,Shape_Leng,Shape_Area,geom
MN,1,301,101,1031703,10317030002,317.03,0001,None,1,0,None,0,0,0,None,None,None,JOE DIMAGGIO HIGHWAY,None,None,None,None,None,None,None,None,None,None,None,U0,07,0,X,UNAVAILABLE OWNER,241719,0,0,0,0,0,0,0,0,0,7,0,0.0,0,0,0.0,0.0,0.0,0.0,None,0,N,0,5,0.0,0.0,0.0,1949,0,0,None,None,0.0,0.0,0.0,0.0,1,1000010301.0,0,031703,978387,199206,12b,None,199 999,10101,None,0.0,None,4,1,1,23v3,t,40.7134492,-74.0211489,None,0.0,241718.831704,b'\x02\x04`\x00\x00\x00\x00\x00\xcb\xa3nI\xa7 BH\x84\x16oI\x80\xf2BH\x02\x00\x00\x00\x01\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\x00\xf0>Z\xad\xe2-A\x00\x80\xfc\xef\x14D\x08A\x00 \xf3ry\xd4-A\x00\xc0}\xa2oZ\x08A\x000\xc6\xdf\xa7\xd4-A\x00\xc0@\xedO^\x08A\x000Wh\xd0\xe2-A\x00\x00\x8c \x07H\x08A\x00\xf0>Z\xad\xe2-A\x00\x80\xfc\xef\x14D\x08A'


In [3]:
%%sql
SELECT count(1) FROM './pluto/MapPLUTO_UNCLIPPED.shp'

Running query in 'duckdb'

count(1)
857036


In [10]:
%%sql 
SELECT * FROM './zoning/nycgiszoningfeatures_202309shp/nyzd.shp' LIMIT 1

Running query in 'duckdb'

ZONEDIST,Shape_Leng,Shape_Area,geom
R4-1,2575.57862978,372245.385615,b'\x02\x04\xe0\x00\x00\x00\x00\x00r\x90oI\xb8\x89#H\xf0\xc8oI\xfbU$H\x02\x00\x00\x00\x01\x00\x00\x00\r\x00\x00\x00\x00\x00\x00\x00\x00V-\xf2\x1d\xf9-A\x00v\x98\xe6$\x81\x04A\x80\x17\xafs\x97\xf6-A\x00\xf8\xbdOdt\x04A\x00\xed\xa0\xffs\xf6-A\x00P\xc5G\xb1s\x04A\x00\xa3\x0f\x0cp\xf6-A\x00\xc8\x11\xc2\x9ds\x04A\x00\xbd\x1f\xfee\xf6-A\x00\xa2\x19\x18ls\x04A\x00<\x9a\x9e\xf3\xf5-A\x00\xc8o\x1e7q\x04A\x00\xb3\xad\x9a!\xf5-A\x00\xcc=\x86\xd3s\x04A\x00\x8d\xe8D\x0e\xf2-A\x00V\x0c\x91\xa6}\x04A\x00\xf0\xb1\x9e\xae\xf2-A\x00\xe0\xee\xb6\xcf\x80\x04A\x00\x8d\xb9\x9e\xbc\xf3-A\x00L\xc6P%\x86\x04A\x80\x0e\x01\xe6\x96\xf4-A\x00\x92%\xd2]\x83\x04A\x00\xe3_\xae\x16\xf6-A\x00\xb4gC\xbf\x8a\x04A\x00V-\xf2\x1d\xf9-A\x00v\x98\xe6$\x81\x04A'


In [5]:
%%sql 
CREATE OR REPLACE TABLE lotzoneper AS
WITH pluto AS (
    select bbl, st_transform(geom, 'EPSG:2263', 'EPSG:4326') geom
    from './pluto/MapPLUTO_UNCLIPPED.shp'
)
, zoningdistricts AS (
    select zonedist, st_transform(geom, 'EPSG:2263', 'EPSG:4326') geom
    from './zoning/nycgiszoningfeatures_202309shp/nyzd.shp'
)
SELECT
    p.bbl,
    n.zonedist,
    SUM(ST_AREA(
        CASE
            WHEN ST_COVEREDBY(p.geom, n.geom) THEN p.geom
            ELSE ST_INTERSECTION(p.geom, n.geom)
        END
    )) AS segbblgeom,
    SUM(ST_AREA(
        CASE
            WHEN ST_COVEREDBY(n.geom, p.geom) THEN n.geom
            ELSE ST_INTERSECTION(n.geom, p.geom)
        END
    )) AS segzonegeom,
    SUM(ST_AREA(p.geom)) AS allbblgeom,
    SUM(ST_AREA(n.geom)) AS allzonegeom
FROM pluto AS p
INNER JOIN zoningdistricts AS n 
    ON ST_INTERSECTS(p.geom, n.geom)
GROUP BY ALL

Running query in 'duckdb'

Count
933549
